In [8]:
pip install pgeocode

In [10]:
pip install requests

In [12]:
pip install matplotlib

In [13]:
import math
import pgeocode
import requests
import json

In [14]:
def cordinatesDistance(lat1, lon1, lat2, lon2) :
    R = 6373.0
    
    lat1 = math.radians(52.2296756)
    lon1 = math.radians(21.0122287)
    lat2 = math.radians(52.406374)
    lon2 = math.radians(16.9251681)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return round(distance)

In [15]:
def getCordinates(srcPincode,desPincode) :
    latitudearr = []
    longitudearr = []
    nomi = pgeocode.Nominatim('in')
    res = nomi.query_postal_code([srcPincode,desPincode])
    latitudearr = res.loc[:,"latitude"]
    longitudearr = res.loc[:,"longitude"]
    return { 'src' : [latitudearr[0],longitudearr[0]],'des' : [latitudearr[1],longitudearr[1]]}

In [16]:
def getPaths(srcPincode,desPincode) :
    path = []
    coordinatesGot = getCordinates(srcPincode,desPincode)
    # print(coordinatesGot)
    url = "https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf624841b747b534394a73b8283ad3de48ff61&start="+str(coordinatesGot['src'][1])+","+str(coordinatesGot['src'][0])+"&"+"end="+str(coordinatesGot['des'][1])+","+str(coordinatesGot['des'][0])
    print(url)
    req = requests.get(url)
    res = json.loads(req.text)
    geometry = res['features'][0]['geometry']
    path = geometry['coordinates']
    return path

In [17]:
clientReq = {
  "Weight": 101,
  "Volume": 123,
  "srcPincode": 605602,
  "desPincode": 600063,
}
Order = [
    {
        "remWeight": 1010,
        "remVolume": 123,
        "srcPincode": 620001,
        "desPincode": 600044,
    }
]

Routes = []
poolArr = []

In [18]:
def pool() :
    clientCoordinates = getCordinates(str(clientReq["srcPincode"]),str(clientReq["desPincode"]))
    for i in range(len(Order)) :
        if clientReq["Weight"] <= Order[i]["remWeight"] and clientReq["Volume"] <= Order[i]["remVolume"] : 
            Routes = getPaths(str(Order[i]['srcPincode']),str(Order[i]['desPincode']))
            sourcePos = -1
            print(clientCoordinates)
            print('-----------------')
            for j in range(len(Routes)) :
                # print('hi')
                if cordinatesDistance(Routes[j][1],Routes[j][0],clientCoordinates['src'][0],clientCoordinates['src'][0]) <= 20 :
                    sourcePos = i
                if cordinatesDistance(Routes[j][1],Routes[j][0],clientCoordinates['des'][0],clientCoordinates['des'][0]) <= 20 :
                    if sourcePos == -1 :
                        break
                    else :
                        poolArr[i] = Order[i]
                        break
    print(poolArr)

In [19]:
pool()

https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf624841b747b534394a73b8283ad3de48ff61&start=78.5278,10.8566&end=80.0591125,12.6668
{'src': [11.9359, 79.5152], 'des': [13.0063, 79.8869]}
-----------------
[]
